1KG
==
Run mushi on 3-SFS computed from 1000 Genome Project data

In [ ]:
%matplotlib inline 
# %matplotlib notebook

from mushi.ksfs import kSFS
import mushi.histories as hst
import mushi.composition as cmp

import numpy as np
import matplotlib as mpl
from collections import OrderedDict
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
import pickle
import copy
import glob
import os

In [ ]:
# set this to e.g. your Downloads folder path if you want plots saved to pdfs
plot_dir = None
# plot_dir = '/Users/williamdewitt/Downloads/'

# data_dir = 'example_data'
data_dir = '../1KG/scons_output/'

# plt.style.use('dark_background')

mpl.rc('text', usetex=True)
mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

Parse population manifest

In [ ]:
pops = {}
for ksfs_file in glob.glob(f'{data_dir}/3-SFS.*.tsv'):
    super_pop, pop = os.path.basename(ksfs_file).split('.')[1:3]
    if super_pop not in pops:
        pops[super_pop] = []        
    if pop not in pops[super_pop]:
        pops[super_pop].append(pop)

pops = OrderedDict((pop, sorted(pops[pop])) for pop in sorted(pops))
pops

### Load 1KG 3-SFSs

In [ ]:
# we use this ordering for triplet mutation types
sorted_triplets = [f'{a5}{a}{a3}>{a5}{d}{a3}' for a in 'AC' for d in 'ACGT' if d != a for a5 in 'ACGT' for a3 in 'ACGT']

ksfs_dict = {}
plt.figure(figsize=(8, 8))
for idx, super_pop in enumerate(pops):
    for pop in pops[super_pop]:
        ksfs_dict[pop] = kSFS(file=f'{data_dir}/3-SFS.{super_pop}.{pop}.tsv')
        ksfs_dict[pop].plot_total(kwargs=dict(ls='', alpha=0.75, marker='o', ms=5, mfc='none', label=pop, c=f'C{idx}'))

        # this sorts the columns of the ksfs according to sorted_triplets
        foo, bar = ksfs_dict[pop].mutation_types.reindex(sorted_triplets)
        ksfs_dict[pop].mutation_types = foo
        ksfs_dict[pop].X = ksfs_dict[pop].X[:, bar]
plt.legend()
plt.show()

Number of segregating variants in each population

In [ ]:
df = pd.DataFrame([(super_pop, pop, ksfs_dict[pop].X.sum()) for super_pop in pops for pop in pops[super_pop]], columns=('superpop', 'pop', 'segregating sites'))
plt.figure(figsize=(12, 5))
sns.barplot(x='pop', y='segregating sites', hue='superpop', data=df, dodge=False)
plt.show()

clip high frequencies due to ancestral state misidentification

In [ ]:
clip_low = 0
clip_high = 20
# we need a different mask vector for each population because the number of haplotypes n
# (length of SFS vector) varies
freq_mask = {}
for super_pop in pops:
    for pop in pops[super_pop]:
        freq_mask[pop] = np.array([True if (clip_low <= i < ksfs_dict[pop].n - clip_high - 1) else False
                                   for i in range(ksfs_dict[pop].n - 1)])

time grid of epoch boundaries (measured in generations)

In [ ]:
change_points = np.logspace(np.log10(1), np.log10(200000), 200)

masked genome size (excluding conserved sites, repeats, 1KG stict mask, and uncertain ancestral states)

In [ ]:
masked_genome_size = pd.read_csv(f'{data_dir}/masked_size.tsv', sep='\t', header=None, names=('context', 'count'))

plt.figure(figsize=(6, 3))
sns.barplot(x='context', y='count', data=masked_genome_size)
plt.xticks(rotation=90)
plt.show()

mutation rate per site per generation

In [ ]:
u = 1.25e-8

mutation rate per masked genome per generation

In [ ]:
mu0 = u * masked_genome_size['count'].sum()
mu0

generation time for time calibration

In [ ]:
t_gen = 29

# Infer $\eta(t)$

regularization paramaters and convergence criteria

In [ ]:
regularization_eta = dict(alpha_tv=1e2, alpha_spline=3e3)
convergence = dict(tol=1e-12, max_iter=2000)

eta_ref = None

for idx_super, super_pop in enumerate(sorted(pops)):
    for idx, pop in enumerate(reversed(pops[super_pop])):
        print(super_pop, pop)
        # clear solutions, in case rerunning this cell
        ksfs_dict[pop].clear_eta()
        ksfs_dict[pop].clear_mu()
        if idx_super == idx == 0:
            alpha_ridge = 1e-10
        else:
            alpha_ridge = 1e4
        ksfs_dict[pop].infer_history(change_points, mu0, eta_ref=eta_ref, alpha_ridge=alpha_ridge,
                                     infer_mu=False,
                                     loss='prf', **regularization_eta,
                                     **convergence, mask=freq_mask[pop])
        if idx_super == idx == 0:
            eta_ref = ksfs_dict[pop].eta

Plot results separately for each superpopulation

In [ ]:
fig, axes = plt.subplots(len(pops), 2, sharex='col', figsize=(6, 1.7 * len(pops)), squeeze=False)
for idx_super, super_pop in enumerate(pops):
    for idx, pop in enumerate(pops[super_pop]):
        plt.sca(axes[idx_super, 0])
        ksfs_dict[pop].plot_total(kwargs=dict(ls='', alpha=0.5, marker='o', ms=5, mfc='none', c=f'C{idx}', label=pop, rasterized=True),
                                  line_kwargs=dict(c=f'C{idx}', ls=':', marker='.', ms=3, alpha=0.5, lw=1, rasterized=True),
                                  fill_kwargs=dict(color=f'C{idx}', alpha=0))
        plt.legend(fontsize=6, loc='upper right')
        if idx_super < len(pops) - 1:
            plt.xlabel(None)
        plt.sca(axes[idx_super, 1])
        ksfs_dict[pop].eta.plot(t_gen=t_gen, lw=2, label=pop, alpha=0.75, c=f'C{idx}')
        plt.xlim([1e3, 1e6])
        plt.legend(fontsize=6, loc='upper right')
        if idx_super < len(pops) - 1:
            plt.xlabel(None)
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/1KG.eta.pdf', dpi=400)
plt.show()

Plot all demographies on the same axes

In [ ]:
fig = plt.figure(figsize=(6, 1.7 * len(pops)))
for idx_super, super_pop in enumerate(pops):
    for idx, pop in enumerate(pops[super_pop]):
        ksfs_dict[pop].eta.plot(t_gen=t_gen, lw=3, label=super_pop if idx == 0 else None, alpha=0.3, c=f'C{idx_super}')
        plt.xlim([1e3, 1e6])
plt.legend()
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/1KG.eta.all.pdf')
plt.show()

# Infer $\boldsymbol\mu(t)$
First we'll focus on sharply timing the Europulse

In [ ]:
import msprime
import stdpopsim
from scipy.interpolate import interp1d

regularization_mu = dict(beta_tv=7e1, beta_ridge=1e-10)

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(17.4, 7.5))

species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model("OutOfAfrica_2T12")
ddb = model.get_demography_debugger()
steps = np.concatenate((np.array([0]), change_points))
eta_Tennessen = hst.eta(change_points,
                        1 / ddb.coalescence_rate_trajectory(steps=steps,
                                                            num_samples=[0, 2],
                                                            double_step_validation=False)[0])

for idx, pop in enumerate(pops['EUR']):
    print(pop)
    eta_mushi = ksfs_dict[pop].eta

    with open(f'Relate_histories/relate_{pop}.coal') as f:
        f.readline()
        t = np.fromstring(f.readline(), sep=' ')
        y = 1 / np.fromstring(f.readline(), sep=' ')[2:]
    t2 = eta_mushi.arrays()[0]
    eta_relate = hst.eta(t2[1:-1], interp1d(t, y, kind='nearest')(t2[:-1]))
    for row_idx, eta in enumerate((eta_Tennessen, eta_relate, eta_mushi)):
        ksfs_dict[pop].clear_mu()
        ksfs_dict[pop].infer_history(change_points, mu0, eta=eta, infer_eta=False,
                                     loss='prf', **regularization_mu,
                                     **convergence, mask=freq_mask[pop])
        plt.sca(axes[row_idx, 0])
        ksfs_dict[pop].plot_total(kwargs=dict(ls='', alpha=0.5, marker='o', ms=5, mfc='none', c=f'C{idx}', label=pop, rasterized=True),
                                  line_kwargs=dict(c=f'C{idx}', ls=':', marker='.', ms=3, alpha=0.5, lw=1, rasterized=True),
                                  fill_kwargs=dict(color=f'C{idx}', alpha=0))
        plt.legend(fontsize=6, loc='upper right')
        if idx_super < len(pops) - 1:
            plt.xlabel(None)

        plt.sca(axes[row_idx, 1])
        ksfs_dict[pop].eta.plot(t_gen=t_gen, lw=2, label=pop, alpha=0.75, c=f'C{idx}')
        plt.xlim([1e3, 1e6])
        plt.legend(fontsize=6, loc='upper right')
        if idx_super < len(pops) - 1:
            plt.xlabel(None)
            
        plt.sca(axes[row_idx, 2])
        ksfs_dict[pop].plot(('TCC>TTC',), clr=True,
                            kwargs=dict(ls='', c=f'C{idx}', marker='o', ms=5, mfc='none', alpha=0.5, label=pop, rasterized=True),
                            line_kwargs=dict(c=f'C{idx}', ls=':', marker='.', ms=3, alpha=0.5, lw=1, rasterized=True))
        plt.ylabel('TCC$\\to$TTC component of\nvariant count composition')
        plt.legend(fontsize=6)

        plt.sca(axes[row_idx, 3])
        plt.gca().set_prop_cycle(None)
        ksfs_dict[pop].mu.plot(('TCC>TTC',), t_gen=t_gen, clr=False, c=f'C{idx}', alpha=0.75, lw=2, label=pop)
        plt.ylabel(r'TCC$\to$TTC mutation intensity')
        plt.xlim([1e3, 1e6])
        plt.legend(fontsize=6)

plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/europulse.pdf', dpi=400)
plt.show()

Now we'll aim for smoother histories that capture all components well

In [ ]:
regularization_mu = dict(hard=True, beta_rank=2e2, beta_tv=0, beta_spline=1e5)
mu_ref = None
for idx_super, super_pop in enumerate(sorted(pops)):    
    for idx, pop in enumerate(reversed(pops[super_pop])):
        print(super_pop, pop)
        ksfs_dict[pop].clear_mu()
        if idx_super == idx == 0:
            beta_ridge = 1e-10
        else:
            beta_ridge = 1e4
        ksfs_dict[pop].infer_history(change_points, mu0, infer_eta=False, mu_ref=mu_ref, beta_ridge=beta_ridge,
                                     loss='prf', **regularization_mu,
                                     **convergence, mask=freq_mask[pop])
        if idx_super == idx == 0:
            mu_ref = ksfs_dict[pop].mu

plot heatmaps of 3-SFS and inferred mush

In [ ]:
with mpl.rc_context(rc={'text.usetex': False}):

    center = 1 / len(sorted_triplets) # for vmin/vmax

    for idx_super, super_pop in enumerate(sorted(pops)):    
        for idx, pop in enumerate(reversed(pops[super_pop])):
            print(pop)
            
            plt.figure(figsize=(8, 10))
            plt.subplot(131)            
            ksfs_dict[pop].plot(clr=True, kwargs=dict(alpha=0.25, ls='', marker='o',
                                                      ms=3, mfc='none', rasterized=True),
                                line_kwargs=dict(ls=':', marker='.', ms=2, alpha=0.25,
                                                 lw=1, rasterized=True))            
            plt.subplot(132)
            ksfs_dict[pop].μ.plot(t_gen=t_gen, clr=True, alpha=0.5, lw=2)
            plt.xscale('log')
            plt.xlim([1e3, 1e6])

            plt.subplot(133)
            σ = np.linalg.svd(cmp.ilr(ksfs_dict[pop].mu.Z), compute_uv=False)
            x = np.arange(1, len(σ) + 1)
            plt.scatter(x, σ)
            plt.plot(x, σ, 'ko', ms=5, mfc='none', mew=.1)
            plt.xscale('log')
            plt.yscale('log')
            plt.xlabel('singular value rank')
            plt.ylabel('singular value')
            plt.tight_layout()
            plt.show()            
            
            singlets = [f'{anc}>{der}' for anc, der in zip(ksfs_dict[pop].mutation_types.str[1],
                                                           ksfs_dict[pop].mutation_types.str[5])]
            
            ksfs_dict[pop].mutation_types.str[1].str.cat(ksfs_dict[pop].mutation_types.str[5], sep='>')
        #     a5 = ksfs_dict[pop].mutation_types.str[0]
        #     a3 = ksfs_dict[pop].mutation_types.str[2]

            col_map = {'A>C':'C0', 'A>G':'C1', 'A>T':'C2', 'C>A':'C3', 'C>G':'C4', 'C>T':'C5'}
            col_colors = [col_map[singlet] for singlet in singlets]

            g = ksfs_dict[pop].clustermap(figsize=(20, 7), col_cluster=False,
                                          xticklabels=True, rasterized=True,
                                          vmin=center / 1.7, vmax=1.7 * center,
                                          cmap='RdBu_r',
                                          col_colors=col_colors)
            g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 9, family='monospace')
        #     g.ax_col_dendrogram.set_visible(False)
            if plot_dir:
                g.savefig(f'{plot_dir}/heatmap.{pop}.X.pdf')
            plt.show()

            g = ksfs_dict[pop].μ.clustermap(t_gen=t_gen,
                                            figsize=(20, 7), col_cluster=False, xticklabels=True, rasterized=True,
                                            vmin=center / 1.2, vmax=1.2 * center,
                                            cmap='RdBu_r',
                                            col_colors=col_colors)
            g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize = 9, family='monospace')
            g.ax_heatmap.set_ylim([172, 58])
        #     g.ax_col_dendrogram.set_visible(False) 
            if plot_dir:
                g.savefig(f'{plot_dir}/heatmap.{pop}.mu.pdf')
            plt.show()

# TMRCA CDF

In [ ]:
fig, axes = plt.subplots(1, len(pops), sharey=True, figsize=(18, 4))
for idx, super_pop in enumerate(pops):
    plt.sca(axes[idx])
    plt.title(super_pop)
    for idx2, pop in enumerate(pops[super_pop]):
        plt.plot(t_gen * ksfs_dict[pop].η.change_points, ksfs_dict[pop].tmrca_cdf(ksfs_dict[pop].eta), label=pop)
        plt.xlabel('$t$')
        plt.ylabel('TMRCA CDF')
        plt.ylim([0, 1])
        plt.xscale('log')
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# truncate to the time range we want to plot

t_gen * np.concatenate(([0], ksfs_dict[pop].μ.change_points))[[58, -30]]

start = 58
end = -30

len(np.concatenate(([0], ksfs_dict[pop].μ.change_points))[58:-30])

In [ ]:
from sklearn.decomposition import PCA
from umap import UMAP

time = np.concatenate(([0], ksfs_dict[pop].μ.change_points))[start:end]

Z_dict = {}
for pop in ksfs_dict:
    Z_dict[pop] = cmp.ilr(ksfs_dict[pop].μ.Z[start:end])
    
Z = np.concatenate(tuple(Z_dict[pop] for pop in ksfs_dict))

np.random.seed(1)

embedding = UMAP(n_components=2,
#                  n_neighbors=50,
#                  min_dist=0,
#                  local_connectivity=1,
#                  n_epochs=1000
                )
# embedding = PCA(n_components=2)

embedding.fit(Z)

# each
Z_transform_dict = {pop:embedding.transform(Z_dict[pop]) for pop in ksfs_dict}


with mpl.rc_context(rc={'text.usetex': False}):

    plt.figure(figsize=(4, 4))
    # plt.subplot(311)
    for idx, super_pop in enumerate(pops):
        for idx2, pop in enumerate(pops[super_pop]):
            plt.plot(*Z_transform_dict[pop].T,
                     '-', lw=3, alpha=.5,
                     c=f'C{idx}',
                     label=super_pop if idx2 == 0 else None)
            plt.annotate(pop, Z_transform_dict[pop][0, :],
                         ha='center', va='center', c='w',
                         family='monospace',
                         bbox=dict(boxstyle='circle', fc=f'C{idx}', ec=f'C{idx}', lw=2),
                         size=6)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')
    plt.legend()
    plt.tight_layout()
    if plot_dir:
        plt.savefig(f'{plot_dir}/1KG.umap.pdf')
    plt.show()

    fig = plt.figure(figsize=(5, 5))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 70)
    # plt.subplot(311)
    for idx, super_pop in enumerate(pops):
        for idx2, pop in enumerate(pops[super_pop]):
            ax.plot(*Z_transform_dict[pop].T, np.log10(t_gen * time),
                    '-', lw=3, alpha=1,
                    c=f'C{idx}'
                    label=super_pop if idx2 == 0 else None)
#             ax.scatter(*Z_transform_dict[pop][None, 0, :].T, np.log10(t_gen * time[1]), s=50,
#                        c=f'C{idx}', alpha=1)
    ax.set_xlabel('UMAP 1')
    ax.set_ylabel('UMAP 2')
    ax.set_zlabel('$\log_{10}(t)$')
    ax.set_zticks([3, 4, 5, 6])
    ax.set_zlim([3, 6])
    # ax.zaxis._set_scale('log')
    # ax.legend()
    plt.tight_layout()
    if plot_dir:
        plt.savefig(f'{plot_dir}/1KG.umap.3D.pdf')
    plt.show()

    fig, axes = plt.subplots(1, 2, sharex='row', figsize=(8,4))
    for umap_comp in (0, 1):
        for idx, super_pop in enumerate(pops):
            for pop in pops[super_pop]:
                axes[umap_comp].plot(Z_transform_dict[pop][:, umap_comp], t_gen * time, 
                         '-', lw=3, alpha=.75,
                         c=f'C{idx}',
                         label=pop)
                axes[umap_comp].annotate(pop, (Z_transform_dict[pop][0, umap_comp], t_gen * time[0]),
                             ha='center', va='center', c='w',
                             family='monospace',
                             bbox=dict(boxstyle='circle', fc=f'C{idx}', ec=f'C{idx}', lw=2),
                             size=6)            
        axes[umap_comp].set_yscale('log')
        axes[umap_comp].set_xlabel(f'UMAP {umap_comp + 1}')
        axes[umap_comp].set_ylabel('$t$ (years ago)')
    # plt.legend(loc=(1.04, -1), fancybox=True, framealpha=0)
    plt.tight_layout()
    if plot_dir:
        plt.savefig(f'{plot_dir}/1KG.umap.comps.pdf')
    plt.show()    

In [ ]:
dmushdt = {}
for idx, super_pop in enumerate(pops):
    for idx2, pop in enumerate(pops[super_pop]):
        t, Z = ksfs_dict[pop].mu.arrays()
        Z = cmp.ilr(Z)
        delta = np.array([Z[i + 1, :] - Z[i, :] for i in range(Z.shape[0] - 1)])
        dmush = np.sqrt((delta ** 2).sum(1))
        dt = np.diff(t[:-1])
        dmushdt[pop] = dmush / (t_gen * dt)

plt.figure(figsize=(2.7, 2))
for idx, super_pop in enumerate(pops):
    for idx2, pop in enumerate(pops[super_pop]):
        plt.plot(t_gen * t[1:-1], (dmushdt[pop] - dmushdt['YRI']) / dmushdt['YRI'],
                 label=super_pop if idx2 == 0 else None, c=f'C{idx}', alpha=0.5, lw=1)
# plt.legend(fontsize=6)
plt.xscale('log')
plt.xlabel(r'$t$ (years ago)')
plt.ylabel(r"$\frac{\|\boldsymbol z'(t)\|_2 - \|\boldsymbol z_0'(t)\|_2}{\|\boldsymbol z_0'(t)\|_2}$")
plt.xlim([1e3, 1e6])
plt.ylim([None, 8])
plt.tight_layout()
if plot_dir:
    plt.savefig(f'{plot_dir}/1KG.drift.pdf')
plt.show()     